In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import metrics
%matplotlib inline

In [16]:
import pickle
import pprint

In [2]:
import autosklearn.classification
import autosklearn.metrics
from autosklearn.metrics import make_scorer

/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [3]:
tf_df = pd.read_csv('../train_features.csv',index_col='sig_id')

In [30]:
tf_df.columns[0:5]

Index(['cp_type', 'cp_time', 'cp_dose', 'g-0', 'g-1'], dtype='object')

In [4]:
pca = PCA(n_components=50)
pca.fit(tf_df.loc[:,'g-0':'c-99'])
pca.explained_variance_ratio_

array([0.41821179, 0.03871707, 0.02358361, 0.02074852, 0.01890126,
       0.01715049, 0.01382442, 0.01341586, 0.00987961, 0.00873211,
       0.00841602, 0.00780523, 0.0066486 , 0.005899  , 0.00580112,
       0.00560802, 0.00517967, 0.0047722 , 0.0047209 , 0.00426508,
       0.00396959, 0.00371703, 0.00340236, 0.00314083, 0.00311071,
       0.00292178, 0.00282075, 0.0026893 , 0.00262019, 0.00252038,
       0.00234773, 0.00226718, 0.00222994, 0.0021396 , 0.00210259,
       0.00200994, 0.00199003, 0.0019211 , 0.00187065, 0.00179917,
       0.00175329, 0.00168097, 0.00165551, 0.00164069, 0.00160625,
       0.00159609, 0.00156018, 0.00154453, 0.0015158 , 0.00146397])

Large drops after 2, 6, 8, 12 components.

In [5]:
reduced_comps = pca.transform(tf_df.loc[:,'g-0':'c-99'])
reduced_comps.shape

(23814, 50)

In [6]:
reduced_comps = reduced_comps[:,:12]
reduced_comps.shape

(23814, 12)

In [7]:
tts_df = pd.read_csv('../train_targets_scored.csv',index_col='sig_id')
moa1 = tts_df.iloc[:,0]
print(moa1.head())

sig_id
id_000644bb2    0
id_000779bfc    0
id_000a6266a    0
id_0015fd391    0
id_001626bd3    0
Name: 5-alpha_reductase_inhibitor, dtype: int64


In [8]:
tts_df.columns

Index(['5-alpha_reductase_inhibitor', '11-beta-hsd1_inhibitor',
       'acat_inhibitor', 'acetylcholine_receptor_agonist',
       'acetylcholine_receptor_antagonist', 'acetylcholinesterase_inhibitor',
       'adenosine_receptor_agonist', 'adenosine_receptor_antagonist',
       'adenylyl_cyclase_activator', 'adrenergic_receptor_agonist',
       ...
       'tropomyosin_receptor_kinase_inhibitor', 'trpv_agonist',
       'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor',
       'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b',
       'vitamin_d_receptor_agonist', 'wnt_inhibitor'],
      dtype='object', length=206)

In [9]:
len(moa1[moa1==1])

17

In [10]:
print("Available CLASSIFICATION metrics autosklearn.metrics.*:")
print("\t" + "\n\t".join(autosklearn.metrics.CLASSIFICATION_METRICS))

Available CLASSIFICATION metrics autosklearn.metrics.*:
	accuracy
	balanced_accuracy
	roc_auc
	average_precision
	log_loss
	precision
	precision_macro
	precision_micro
	precision_samples
	precision_weighted
	recall
	recall_macro
	recall_micro
	recall_samples
	recall_weighted
	f1
	f1_macro
	f1_micro
	f1_samples
	f1_weighted


In [19]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=360,
    per_run_time_limit=30,
    max_models_on_disc=None,
    ensemble_memory_limit=1.75*1024,
    ml_memory_limit=3.5*1024,
    tmp_folder='./ask_tmp',
    output_folder='./ask_out',
    delete_output_folder_after_terminate=True,
    n_jobs=2,
    metric=autosklearn.metrics.log_loss
)

In [20]:
automl.fit(reduced_comps, moa1)

/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


AutoSklearnClassifier(dask_client=None,
                      delete_output_folder_after_terminate=True,
                      delete_tmp_folder_after_terminate=True,
                      disable_evaluator_output=False,
                      ensemble_memory_limit=1792.0, ensemble_nbest=50,
                      ensemble_size=50, exclude_estimators=None,
                      exclude_preprocessors=None, get_smac_object_callback=None,
                      include_estimators=None, include_preprocessors=None,
                      initial_configurations_via_metalearning=25,
                      logging_config=None, max_models_on_disc=None,
                      metadata_directory=None, metric=None,
                      ml_memory_limit=3584.0, n_jobs=2,
                      output_folder='./ask_out', per_run_time_limit=30,
                      resampling_strategy='holdout',
                      resampling_strategy_arguments=None, seed=1,
                      smac_scenario_args=None,

In [21]:
print(automl.show_models())

[(0.720000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'classifier:__choice__': 'gradient_boosting', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'quantile_transformer', 'feature_preprocessor:__choice__': 'feature_agglomeration', 'classifier:gradient_boosting:early_stop': 'off', 'classifier:gradient_boosting:l2_regularization': 0.6830439907299465, 'classifier:gradient_boosting:learning_rate': 0.1132047541281332, 'classifier:gradient_boosting:loss': 'auto', 'classifier:gradient_boosting:max_bins': 255, 'classifier:gradient_boosting:max_depth': 'None', 'classifier:gradient_boosting:max_leaf_nodes': 309, 'classifier:gradient_boosting:min_samples_leaf': 72, 'classifier:gradient_boosting:scori

In [22]:
model_suite = automl.get_models_with_weights()
model_suite[0]

(0.72,
 SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'classifier:__choice__': 'gradient_boosting', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'quantile_transformer', 'feature_preprocessor:__choice__': 'feature_agglomeration', 'classifier:gradient_boosting:early_stop': 'off', 'classifier:gradient_boosting:l2_regularization': 0.6830439907299465, 'classifier:gradient_boosting:learning_rate': 0.1132047541281332, 'classifier:gradient_boosting:loss': 'auto', 'classifier:gradient_boosting:max_bins': 255, 'classifier:gradient_boosting:max_depth': 'None', 'classifier:gradient_boosting:max_leaf_nodes': 309, 'classifier:gradient_boosting:min_samples_leaf': 72, 'classifier:gradient_boosting:scoring':

In [23]:
len(model_suite)

5

In [24]:
predictions = automl.predict(reduced_comps)

In [25]:
metrics.log_loss(moa1,predictions)

0.008702135650016285

In [26]:
np.sum(predictions)

11

In [27]:
np.sum(np.array(moa1))

17

In [49]:
automl_all = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=6000,
    per_run_time_limit=300,
    max_models_on_disc=None,
    ensemble_memory_limit=1.75*1024,
    ml_memory_limit=3.5*1024,
    tmp_folder=None,
    output_folder='./autosklearn_output',
    delete_output_folder_after_terminate=True,
    n_jobs=2,
    metric=autosklearn.metrics.log_loss
)

In [50]:
automl_all.fit(reduced_comps,tts_df)

[WARNING] [2020-11-02 17:01:43,357:EnsembleBuilder(1):a885797645e95656a7e609cc6f2020a5] Memory Exception -- restart with less ensemble_nbest: 25
[WARNING] [2020-11-02 17:20:46,718:EnsembleBuilder(1):a885797645e95656a7e609cc6f2020a5] Memory Exception -- restart with less ensemble_nbest: 12
[WARNING] [2020-11-02 17:25:35,186:EnsembleBuilder(1):a885797645e95656a7e609cc6f2020a5] Memory Exception -- restart with less ensemble_nbest: 6
[WARNING] [2020-11-02 17:30:45,659:EnsembleBuilder(1):a885797645e95656a7e609cc6f2020a5] Memory Exception -- restart with less ensemble_nbest: 3
[WARNING] [2020-11-02 17:36:33,649:EnsembleBuilder(1):a885797645e95656a7e609cc6f2020a5] Memory Exception -- restart with less ensemble_nbest: 1


/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


AutoSklearnClassifier(dask_client=None,
                      delete_output_folder_after_terminate=True,
                      delete_tmp_folder_after_terminate=True,
                      disable_evaluator_output=False,
                      ensemble_memory_limit=1792.0, ensemble_nbest=50,
                      ensemble_size=50, exclude_estimators=None,
                      exclude_preprocessors=None, get_smac_object_callback=None,
                      include_estimators=None, include_preprocessors=None,
                      initial_configurations_via_metalearning=25,
                      logging_config=None, max_models_on_disc=None,
                      metadata_directory=None, metric=None,
                      ml_memory_limit=3584.0, n_jobs=2,
                      output_folder='./autosklearn_output',
                      per_run_time_limit=300, resampling_strategy='holdout',
                      resampling_strategy_arguments=None, seed=1,
                      smac_scenari

In [51]:
with open('automodel.pkl','wb') as cellar:
    pickle.dump(automl_all,cellar)

In [12]:
with open('automodel.pkl','rb') as cellar:
    automl_all = pickle.load(cellar)

In [14]:
pred_all = automl_all.predict(reduced_comps)
metrics.log_loss(tts_df,pred_all)

4.144142301877526

In [17]:
pprint.pprint(metrics.classification_report(tts_df,pred_all,output_dict=True))

{'0': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 17},
 '1': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 18},
 '10': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 360},
 '100': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 18},
 '101': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 60},
 '102': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 72},
 '103': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 106},
 '104': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 59},
 '105': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 241},
 '106': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 24},
 '107': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 32},
 '108': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 72},
 '109': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 283},
 '11': {'f1-score': 0.0, '

In [18]:
automl_all.show_models()

"[(1.000000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'classifier:__choice__': 'bernoulli_nb', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'median', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'standardize', 'feature_preprocessor:__choice__': 'feature_agglomeration', 'classifier:bernoulli_nb:alpha': 2.9559216410631404, 'classifier:bernoulli_nb:fit_prior': 'True', 'data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction': 0.03863647861706608, 'feature_preprocessor:feature_agglomeration:affinity': 'euclidean', 'feature_preprocessor:feature_agglomeration:linkage': 'average', 'feature_preprocessor:feature_agglomeration:n_clusters': 368, 'feature_preprocessor:feature_agglomeration:pooling_

That did not do so well across the whole target space, did it?

It is just too interesting... I will try running a model for each column. Also, I will use the whole input dataframe and let autosklearn one hot encode the two binary columns in addition to standardizing the treatment duration column.

In [33]:
bespoke_models = []
bespoke_scores = []
tf_df['cp_type']=tf_df['cp_type'].astype('category')
tf_df['cp_dose']=tf_df['cp_dose'].astype('category')
for i in range(len(tts_df.columns)):
    bespoke = autosklearn.classification.AutoSklearnClassifier(
        time_left_for_this_task=360,
        per_run_time_limit=30,
        max_models_on_disc=None,
        ensemble_memory_limit=1.75*1024,
        ml_memory_limit=3.5*1024,
        tmp_folder=None,
        output_folder='./bespoke_out'+str(i),
        delete_output_folder_after_terminate=True,
        n_jobs=2,
        metric=autosklearn.metrics.log_loss
    )
    bespoke.fit(tf_df,tts_df.iloc[:,i])
    bespoke_models.append(bespoke)
    bespoke_scores.append(metrics.log_loss(tts_df.iloc[:,i],bespoke.predict(tf_df)))
with open('bespokemodels.pkl','wb') as cellar:
    pickle.dump([bespoke_models,bespoke_scores],cellar)
print(np.mean(np.array(bespoke_scores)))

Exception ignored in: <function BackendContext.__del__ at 0x7fd24c9e5280>
Traceback (most recent call last):
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/util/backend.py", line 121, in __del__
    self.delete_directories(force=False)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/util/backend.py", line 126, in delete_directories
    raise ValueError("Failed to delete output dir: %s because auto-sklearn did not "
ValueError: Failed to delete output dir: ./bespoke_out because auto-sklearn did not create it. Please make sure that the specified output dir does not exist when instantiating auto-sklearn.


[WARNING] [2020-11-03 17:37:53,239:EnsembleBuilder(1):7072454b0b93a3378bd7012e62c70998] Error loading /tmp/autosklearn_tmp_30cceb2e-1e25-11eb-ab65-b3ba7ff21120/.auto-sklearn/predictions_ensemble/predictions_ensemble_1_1_0.0.npy: Traceback (most recent call last):
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/ensemble_builder.py", line 508, in score_ensemble_preds
    score = calculate_score(solution=self.y_true_ensemble,
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 375, in calculate_score
    score = metric(solution, prediction)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 135, in __call__
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 2253, in log_loss
    raise ValueError('y_true con

[WARNING] [2020-11-03 17:38:07,440:EnsembleBuilder(1):7072454b0b93a3378bd7012e62c70998] Error loading /tmp/autosklearn_tmp_30cceb2e-1e25-11eb-ab65-b3ba7ff21120/.auto-sklearn/predictions_ensemble/predictions_ensemble_1_1_0.0.npy: Traceback (most recent call last):
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/ensemble_builder.py", line 508, in score_ensemble_preds
    score = calculate_score(solution=self.y_true_ensemble,
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 375, in calculate_score
    score = metric(solution, prediction)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 135, in __call__
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 2253, in log_loss
    raise ValueError('y_true con

[WARNING] [2020-11-03 17:38:23,560:EnsembleBuilder(1):7072454b0b93a3378bd7012e62c70998] Error loading /tmp/autosklearn_tmp_30cceb2e-1e25-11eb-ab65-b3ba7ff21120/.auto-sklearn/predictions_ensemble/predictions_ensemble_1_1_0.0.npy: Traceback (most recent call last):
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/ensemble_builder.py", line 508, in score_ensemble_preds
    score = calculate_score(solution=self.y_true_ensemble,
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 375, in calculate_score
    score = metric(solution, prediction)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 135, in __call__
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 2253, in log_loss
    raise ValueError('y_true con

[WARNING] [2020-11-03 17:38:39,682:EnsembleBuilder(1):7072454b0b93a3378bd7012e62c70998] Error loading /tmp/autosklearn_tmp_30cceb2e-1e25-11eb-ab65-b3ba7ff21120/.auto-sklearn/predictions_ensemble/predictions_ensemble_1_1_0.0.npy: Traceback (most recent call last):
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/ensemble_builder.py", line 508, in score_ensemble_preds
    score = calculate_score(solution=self.y_true_ensemble,
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 375, in calculate_score
    score = metric(solution, prediction)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 135, in __call__
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 2253, in log_loss
    raise ValueError('y_true con

...

[WARNING] [2020-11-03 17:42:43,348:EnsembleBuilder(1):7072454b0b93a3378bd7012e62c70998] Error loading /tmp/autosklearn_tmp_30cceb2e-1e25-11eb-ab65-b3ba7ff21120/.auto-sklearn/predictions_ensemble/predictions_ensemble_1_1_0.0.npy: Traceback (most recent call last):
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/ensemble_builder.py", line 508, in score_ensemble_preds
    score = calculate_score(solution=self.y_true_ensemble,
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 375, in calculate_score
    score = metric(solution, prediction)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 135, in __call__
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 2253, in log_loss
    raise ValueError('y_true con

[WARNING] [2020-11-03 17:42:59,469:EnsembleBuilder(1):7072454b0b93a3378bd7012e62c70998] Error loading /tmp/autosklearn_tmp_30cceb2e-1e25-11eb-ab65-b3ba7ff21120/.auto-sklearn/predictions_ensemble/predictions_ensemble_1_1_0.0.npy: Traceback (most recent call last):
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/ensemble_builder.py", line 508, in score_ensemble_preds
    score = calculate_score(solution=self.y_true_ensemble,
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 375, in calculate_score
    score = metric(solution, prediction)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 135, in __call__
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 2253, in log_loss
    raise ValueError('y_true con

[WARNING] [2020-11-03 17:43:15,581:EnsembleBuilder(1):7072454b0b93a3378bd7012e62c70998] Error loading /tmp/autosklearn_tmp_30cceb2e-1e25-11eb-ab65-b3ba7ff21120/.auto-sklearn/predictions_ensemble/predictions_ensemble_1_1_0.0.npy: Traceback (most recent call last):
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/ensemble_builder.py", line 508, in score_ensemble_preds
    score = calculate_score(solution=self.y_true_ensemble,
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 375, in calculate_score
    score = metric(solution, prediction)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 135, in __call__
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 2253, in log_loss
    raise ValueError('y_true con

[WARNING] [2020-11-03 17:43:31,723:EnsembleBuilder(1):7072454b0b93a3378bd7012e62c70998] Error loading /tmp/autosklearn_tmp_30cceb2e-1e25-11eb-ab65-b3ba7ff21120/.auto-sklearn/predictions_ensemble/predictions_ensemble_1_1_0.0.npy: Traceback (most recent call last):
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/ensemble_builder.py", line 508, in score_ensemble_preds
    score = calculate_score(solution=self.y_true_ensemble,
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 375, in calculate_score
    score = metric(solution, prediction)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/autosklearn/metrics/__init__.py", line 135, in __call__
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/paulus/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 2253, in log_loss
    raise ValueError('y_true con

ValueError: No valid model found in run history. This means smac was not able to fit a valid model. Please check the log file for errors.

Broken piece of trash. Let's give up on this and if I feel the need, do a logistic regression on each individual target for a baseline / identification of problem columns. I could do a RegressorChain if I thought that there would be any benefit to letting the target columns talk to each other.